<a href="https://colab.research.google.com/github/QuangHuy54/chatbot-shipping-status/blob/main/shipping_status_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Shipping status chatbot

## 1. Database

I deployed the database on Google BigQuery. These steps will show an overview of the database.

In [ ]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [ ]:
project_id = 'chatbot-437514'

# Package used for interfacing w/ BigQuery from Python
from google.cloud import bigquery

# Create BigQuery client
bq_client = bigquery.Client(project = project_id)

Table USER:

In [ ]:
user_sql = """
    SELECT *
    FROM `chatbot-437514.E_commerce.USER`
    ORDER BY USER_ID
"""
df = bq_client.query_and_wait(user_sql).to_dataframe()
df

,USER_ID,NAME
0,0001,Nguyen
1,0002,Quang
2,0003,Huy


Table PRODUCT:

In [ ]:
product_sql = """
    SELECT *
    FROM `chatbot-437514.E_commerce.PRODUCT`
    ORDER BY PRODUCT_ID
"""
df = bq_client.query_and_wait(product_sql).to_dataframe()
df

,PRODUCT_ID,PRODUCT_NAME,SUPPLIER_NAME,SUPPLIER_ADDRESS
0,0001,Linear algebra book,Fahasa bookstore,"Nguyen Duy Trinh, District 2"
1,0002,Machine learning book,Fahasa bookstore,"Nguyen Duy Trinh, District 2"
2,0003,Laptop Asus,Phong Vu Computer,"Nguyen Thi Thap, District 7"
3,0004,Laptop Vivo,Phong Vu Computer,"Nguyen Thi Thap, District 7"
4,0005,Iphone 15,Thegioididong,"Nguyen Thi Dinh, District 2"
5,0006,Samsung 22,Thegioididong,"Nguyen Thi Dinh, District 2"
6,0007,Deep learning book,Fahasa bookstore,"Nguyen Duy Trinh, District 2"


Table ORDER and ORDER_PRODUCT:

In [ ]:
order_sql = """
    SELECT *
    FROM `chatbot-437514.E_commerce.ORDER` INNER JOIN `chatbot-437514.E_commerce.ORDER_PRODUCT` USING (ORDER_ID)
"""
df = bq_client.query_and_wait(order_sql).to_dataframe()
df

,ORDER_ID,USER_ID,ORDER_DATE,STATUS,ADDRESS,PRICE,TRACKING_ID,SHIPPER_ID,PRODUCT_ID,QUANTITY
0,0001,0001,2024-11-04,To Pay,"The Sun Avenue, District 2",1200,0001,0001,0002,4
1,0001,0001,2024-11-04,To Pay,"The Sun Avenue, District 2",1200,0001,0001,0001,3
2,0002,0001,2024-11-02,To Ship,"The Sun Avenue, District 2",1300,0002,0002,0005,10
3,0004,0002,2024-11-11,To Ship,"Sadora Apartment, Thu Duc District",2000,0004,0002,0007,1
4,0008,0001,2024-11-05,To Receive,"The Sun Avenue, District 2",3000,0008,0003,0001,2
5,0003,0001,2024-11-05,To Receive,"The Sun Avenue, District 2",1500,0003,0003,0003,4
6,0005,0002,2024-11-12,To Ship,"Sadora Apartment, Thu Duc District",2300,0005,0003,0006,2
7,0006,0003,2024-11-06,To Receive,"New City Thu Thiem, District 2",3000,0006,0001,0003,3
8,0003,0001,2024-11-05,To Receive,"The Sun Avenue, District 2",1500,0003,0003,0006,1
9,0007,0003,2024-11-20,To Pay,"New City Thu Thiem, District 2",3000,0007,0001,0004,2


Table SHIPPER

In [ ]:
shipper_sql = """
    SELECT *
    FROM `chatbot-437514.E_commerce.SHIPPER`
"""
df = bq_client.query_and_wait(shipper_sql).to_dataframe()
df

,SHIPPER_ID,NAME,PHONE_NUMBER
0,0003,Duong Qua C,0934411222
1,0002,Luong Van B,0935211222
2,0001,Nguyen Van A,0933111222


## 2. Create available tools for the chatbot


Define functions (I just define functions for three cases)

In [ ]:
!pip install thefuzz

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 25.8 MB/s eta 0:00:00


In [ ]:
from thefuzz import fuzz

def get_orders_id(order_ids,dates,product_names,user_id):
  if order_ids is not None:
    return order_ids
  if dates is None and product_names is None:
    query = """
        SELECT ORDER_ID
        FROM `chatbot-437514.E_commerce.ORDER`
        WHERE USER_ID = '{}'
    """.format(user_id)
    result=bq_client.query_and_wait(query).to_dataframe()
    return result['ORDER_ID'].values.tolist()
  result_by_date=None
  result_by_product=None
  if dates is not None:
    for date in dates:
      query = """
          SELECT ORDER_ID
          FROM `chatbot-437514.E_commerce.ORDER`
          WHERE USER_ID = '{}' AND ORDER_DATE = '{}'
      """.format(user_id,date)
      result=bq_client.query_and_wait(query).to_dataframe()
      if result_by_date is None:
        result_by_date=set(result['ORDER_ID'].values.tolist())
      else:
        result_by_date=result_by_date.union(result['ORDER_ID'].values.tolist())
  if product_names is not None:
      query=  """
          SELECT ORDER_ID, PRODUCT_ID,PRODUCT_NAME
          FROM (`chatbot-437514.E_commerce.ORDER` INNER JOIN `chatbot-437514.E_commerce.ORDER_PRODUCT` USING (ORDER_ID)) INNER JOIN `chatbot-437514.E_commerce.PRODUCT` USING (PRODUCT_ID)
          WHERE USER_ID = '{}'
      """.format(user_id)
      result=bq_client.query_and_wait(query).to_dataframe()
      products=set(result['PRODUCT_NAME'].values.tolist())
      for name in product_names:
        similarities={name:fuzz.ratio(name,product_name) for product_name in products}
        best_matching_name=max(similarities, key = lambda x: similarities[x])
        if similarities[best_matching_name]>=0.8:
          result_by_name=result[result['PRODUCT_NAME']==best_matching_name]
          if result_by_product is None:
            result_by_product=set(result_by_name['ORDER_ID'].values.tolist())
          else:
            result_by_product=result_by_product.union(result_by_name['ORDER_ID'].values.tolist())
  if result_by_date is not None and result_by_product is not None:
    return result_by_date.intersection(result_by_product)
  else:
    return result_by_date if result_by_date is not None else result_by_product

def get_location(tracking_id):
  #This is just an example for visualization, real result would be coordinates not those strings.
  res={'0001':'Binh Duong','0002':'Ha Noi','0003':'Ho Chi Minh','0004':'Da Lat','0005':'Dong Nai','0006':'Can Tho','0007':'Ca Mau'}
  return res[tracking_id]

def get_shipping_status(order_ids,dates,product_names,user_id):
  orders=get_orders_id(order_ids,dates,product_names,user_id)
  if orders is None:
    return None
  result=None
  for order in orders:
    query = """
        SELECT ORDER_ID, STATUS,PRICE,TRACKING_ID,ORDER_DATE,NAME,PHONE_NUMBER
        FROM `chatbot-437514.E_commerce.ORDER` INNER JOIN `chatbot-437514.E_commerce.SHIPPER` USING (SHIPPER_ID)
        WHERE ORDER_ID = '{}'
    """.format(order)
    result_order=bq_client.query_and_wait(query).to_dataframe()
    status=result_order['STATUS'].values.tolist()[0]
    res_dict={}
    if status=='To Pay':
      res_dict={'order_id':order,'status':'To Pay','price':result_order['PRICE'].values.tolist()[0]}
    elif status=='To Ship':
      res_dict={'order_id':order,'status':'To Ship','current_location':get_location(result_order['TRACKING_ID'].values.tolist()[0])}
    else:
      res_dict={'order_id':order,'status':'To Receive','shipper_name':result_order['NAME'].values.tolist()[0],'shipper_phone_number':result_order['PHONE_NUMBER'].values.tolist()[0]}
    if result is None:
      result=res_dict
    else:
      result.append(res_dict)
  return result

def get_current_position(order_ids,dates,product_names,user_id):
    orders=get_orders_id(order_ids,dates,product_names,user_id)
    if orders is None:
      return None
    else:
      result=[]
      for order in orders:
          query = """
              SELECT ORDER_ID,NAME,TRACKING_ID
              FROM `chatbot-437514.E_commerce.ORDER`
              WHERE ORDER_ID = '{}'
          """.format(order)
          result_order=bq_client.query_and_wait(query).to_dataframe()
          res_dict={'order_id':order,'current_location':get_location(result_order['TRACKING_ID'].values.tolist()[0])}
          if result is None:
            result=res_dict
          else:
            result.append(res_dict)
      return result
def get_shipper_information(order_ids,dates,product_names,user_id):
    orders=get_orders_id(order_ids,dates,product_names,user_id)
    if orders is None:
      return None
    else:
      result=[]
      for order in orders:
          query = """
              SELECT ORDER_ID,NAME,PHONE_NUMBER
              FROM `chatbot-437514.E_commerce.ORDER` INNER JOIN `chatbot-437514.E_commerce.SHIPPER` USING (SHIPPER_ID)
              WHERE ORDER_ID = '{}'
          """.format(order)
          result_order=bq_client.query_and_wait(query).to_dataframe()
          res_dict={'order_id':order,'shipper_name':result_order['NAME'].values.tolist()[0],'shipper_phone_number':result_order['PHONE_NUMBER'].values.tolist()[0]}
          if result is None:
            result=res_dict
          else:
            result.append(res_dict)
      return result

Define tools for LLM Model

In [ ]:
import vertexai
from vertexai.generative_models import (
    FunctionDeclaration,
    GenerationConfig,
    GenerativeModel,
    Part,
    Tool,
)

get_shipping_status_func=FunctionDeclaration(
    name="get_shipping_status",
    description="Retrieve the the shipper information based on orders information.",
    parameters={
        "type": "object",
        "properties": {
            "order_ids":{"type": "array", "description": "List of order ids that user want to retrieve. Set to None if users don't provide spcific ids."},
            "dates":{"type": "array", "description": "List of dates that user want to retrieve orders on that day. The date has a form 'YYYY-MM-DD' and remember user usally provide the form 'DD/MM/YYYY'. Set to None if users don't provide spcific dates."},
            "product_names":{"type": "array", "description":"List of product names that user want to retrieve orders contain them. Set to None if users don't provide spcific product names."},
        },
        "required":['order_ids','dates','product_names']
    },
)
get_current_position_func=FunctionDeclaration(
    name="get_current_position",
    description="Retrieve the the current position of the orders.",
    parameters={
        "type": "object",
        "properties": {
            "order_ids":{"type": "array", "description": "List of order ids that user want to retrieve. Set to None if users don't provide spcific ids."},
            "dates":{"type": "array", "description": "List of dates that user want to retrieve orders on that day. The date has a form 'YYYY-MM-DD' and remember user usually provide the form 'DD/MM/YYYY'. Set to None if users don't provide spcific dates."},
            "product_names":{"type": "array", "description":"List of product names that user want to retrieve orders contain them. Set to None if users don't provide spcific product names."},
        },
        "required":['order_ids','dates','product_names']
    },
)
get_shipping_status_func=FunctionDeclaration(
    name="get_shipping_status",
    description="Retrieve the the shipping status of the orders. For shipping status is To Pay, the order price are also responsed to the user. For shipping status is To Ship,  the order's current location are also responsed to the user. For shipping status is To Receive, the information of the shipper are also responsed to the user.",
    parameters={
        "type": "object",
        "properties": {
            "order_ids":{"type": "array", "description": "List of order ids that user want to retrieve. Set to None if users don't provide spcific ids."},
            "dates":{"type": "array", "description": "List of dates that user want to retrieve orders on that day. The date has a form 'YYYY-MM-DD'. Set to None if users don't provide spcific dates."},
            "product_names":{"type": "array", "description":"List of product names that user want to retrieve orders contain them. Set to None if users don't provide spcific product names."},
        },
        "required":['order_ids','dates','product_names']
    },
)
tools = Tool(
    function_declarations=[
        get_shipping_status_func,
        get_current_position_func,
        get_shipping_status_func,
    ],
)


## 3. Building chatbot


Define Gemini Model using Vertex AI

In [ ]:
import vertexai
from vertexai.generative_models import (
    FunctionDeclaration,
    GenerationConfig,
    GenerativeModel,
    Part,
    GenerationResponse,
    Content,
    Tool,
)

PROJECT_ID = "chatbot-437514"
LOCATION = "us-central1"
vertexai.init(project=PROJECT_ID, location=LOCATION)
model = GenerativeModel(
    "gemini-1.5-pro-001",
    generation_config=GenerationConfig(temperature=0),
    tools=[tools],
)

Define some support functions:

In [ ]:
user_id='0001'
vertexai.init(project=PROJECT_ID, location=LOCATION)

model = GenerativeModel(
    "gemini-1.5-pro-001",
    generation_config=GenerationConfig(temperature=0),
    tools=[tools],
)
def extract_function_calls(response: GenerationResponse) -> list[dict]:
    function_calls: list[dict] = []
    if response.candidates[0].function_calls:
        for function_call in response.candidates[0].function_calls:
            function_call_dict: dict[str, dict[str, Any]] = {function_call.name: {}}
            for key, value in function_call.args.items():
                function_call_dict[function_call.name][key] = value
            function_calls.append(function_call_dict)
    return function_calls

def handle_message(function_calls):
    result=[]
    for function_call in function_calls:
        function_name=list(function_call.keys())[0]
        try:
            args=function_call[function_name]
            if function_name=="get_shipping_status":
                result.append(Part.from_function_response(
                    name="get_shipping_status",
                    response={
                        "content":get_shipping_status(args['order_ids'],args['dates'],args['product_names'],user_id),
                    },))
            elif function_name=="get_current_position":
                result.append(Part.from_function_response(
                    name="get_current_position",
                    response={
                        "content":get_current_position(args['order_ids'],args['dates'],args['product_names'],user_id),
                    },))
            elif function_name=="get_shipping_status":
                args=function_call[function_name]
                result.append(Part.from_function_response(
                    name="get_shipping_status",
                    response={
                        "content":get_shipping_status(args['order_ids'],args['dates'],args['product_names'],user_id),
                },))
        except IndexError as e:
           print(f"Error processing item: {e}")
    return result

Building conversation

In [ ]:
context="You are a chat assistant which is integrated into an E-commerce system to answer about shipping status or user's orders. The assistant should: Provide clear, accurate information based on supplied tools; Be transparent when an answer is not known or when the necessary information is not available, stating that it cannot provide speculative or incomplete answers;Maintain a helpful, polite and professional tone, keeping the focus on answering user's questions; Avoid asking another questions after providing an answer; If a user requests information or assistance that the assistant cannot directly provide, it should avoid them and tell customers the problem."
greeting="What can I help with ?"
chat=model.start_chat(history=[Content(role='user',parts=[Part.from_text(context)]),Content(role='model',parts=[Part.from_text(greeting)])])

Testing different situations:

In [ ]:
prompt="Can you tell me what is the shipping status of the order I made at 4/11/2024 ?"
print("User: ",prompt)
response_model = chat.send_message(prompt)
function_calls = extract_function_calls(response_model)
if len(function_calls)==0:
  print("Chatbot: ",response_model.text)
else:
  response=handle_message(function_calls)
  final_response = chat.send_message(response)
  print("Chatbot: ",final_response.text)

User:  Can you tell me what is the shipping status of the order I made at 4/11/2024 ?
Chatbot:  The order's status is To Pay and the price is 1200. 



In [ ]:
prompt="Can you tell me what is the shipping status of the Iphone 15 I ordered?"
print("User: ",prompt)
response_model = chat.send_message(prompt)
function_calls = extract_function_calls(response_model)
if len(function_calls)==0:
  print("Chatbot: ",response_model.text)
else:
  response=handle_message(function_calls)
  final_response = chat.send_message(response)
  print("Chatbot: ",final_response.text)

User:  Can you tell me what is the shipping status of the Iphone 15 I ordered?
Chatbot:  The order's shipping status is To Ship and its current location is Ha Noi. 



In [ ]:
prompt="Can you give me the shipper information of the order 0003?"
print("User: ",prompt)
response_model = chat.send_message(prompt)
function_calls = extract_function_calls(response_model)
if len(function_calls)==0:
  print("Chatbot: ",response_model.text)
else:
  response=handle_message(function_calls)
  final_response = chat.send_message(response)
  print("Chatbot: ",final_response.text)

User:  Can you give me the shipper information of the order 0003?
Chatbot:  The order's shipping status is To Receive. The shipper information is shipper name: Duong Qua C, shipper phone number: 0934411222. 

